# Weather Bot

## Testing gemini API key

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os 
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [3]:
response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
logger.info(response.text)

The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's a breakdown:

*   **Sunlight is a mixture of all colors:** Sunlight appears white to our eyes, but it's actually composed of all the colors of the rainbow (red, orange, yellow, green, blue, indigo, violet).

*   **Sunlight enters the Earth's atmosphere:** When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).

*   **Scattering of light:** This collision causes the light to scatter in different directions.

*   **Rayleigh scattering favors shorter wavelengths:** Rayleigh scattering is more effective at scattering shorter wavelengths of light. Blue and violet light have shorter wavelengths than other colors like red and orange.

*   **Why not violet?** While violet light is scattered even more than blue light, there are a couple of reasons why the sky appears blue:

    *   **Sunlight emits less violet light:** The sun emits less violet light compared to 

# Testing Crew AI

## 0. Imports

In [4]:
import os

os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## 1. Tools Initialization

### Initialize built-in tools

In [5]:
from crewai_tools import SerperDevTool

serper_tool = SerperDevTool()

### Initialize custom tools

In [6]:
# Initialize custom tools
from crewai.tools import tool
import requests

@tool("my_simple_tool")
def my_simple_tool(location: str) -> str:
    """Get current weather for a city"""

    api_key = os.getenv("WEATHER_API_KEY")
    base_url = "http://api.weatherapi.com/v1/current.json"
    
    try:
        # Make the API request
        response = requests.get(
            f"{base_url}?key={api_key}&q={location}"
        )
        data = response.json()

        # Extract and format weather details
        weather_info = f"""
        Weather in {data['location']['name']}:
        Country: {data['location']['country']}
        Region: {data['location']['region']}
        Latitude: {data['location']['lat']}, Longitude: {data['location']['lon']}
        Local-Time: {data['location']['localtime']}

        Condition: {data['current']['condition']['text']}
        Temperature: {data['current']['temp_c']}°C / {data['current']['temp_f']}°F
        Feels Like: {data['current']['feelslike_c']}°C / {data['current']['feelslike_f']}°F
        Humidity: {data['current']['humidity']}%
        Cloud Cover: {data['current']['cloud']}%
        Precipitation: {data['current']['precip_mm']} mm
        Wind Speed: {data['current']['wind_kph']} km/h ({data['current']['wind_mph']} mph)
        Wind Direction: {data['current']['wind_dir']} ({data['current']['wind_degree']}°)
        Gust Speed: {data['current']['gust_kph']} km/h ({data['current']['gust_mph']} mph)

        Visibility: {data['current']['vis_km']} km
        UV Index: {data['current']['uv']}
        Pressure: {data['current']['pressure_mb']} mb / {data['current']['pressure_in']} in

        Weather Updated At: {data['current']['last_updated']}
        """
        return weather_info
    except Exception as e:
        return f"Error fetching weather data: {str(e)}"



## 2. Create Agents

In [7]:
from crewai import Agent

# Weather Agent
weather_agent = Agent(
    name="WeatherBot",
    role="Weather Reporter",
    goal="Provide weather details of the {location} using WeatherAPI.",
    backstory="An AI assistant specializing in providing weather updates and forecasts.",
    verbose=True,
    allow_delegation=False,
    llm="gemini/gemini-1.5-flash",
    tools=[my_simple_tool]  # Attach WeatherAPI tool
)

# Search Agent
search_agent = Agent(
    name="SearchBot",
    role="Human Habitant Researcher",
    goal="Search for information for Today's weather conditions using Serper API and recommed the correct living style. Please Use all the details provided by WeatherBot.",
    backstory="An Human Habitant Researcher to perform online searches and gather information and suggest good life style.",
    verbose=True,
    allow_delegation=False,
    llm="gemini/gemini-1.5-flash",
    tools=[serper_tool]  # Attach Serper API tool
)

# Writer Agent
writer_agent = Agent(
    name="WriterBot",
    role="Content Writer",
    goal="Write a comprehensive weather report (Today' s) based on data collected by other agents. And life style for human according to the weather.",
    backstory="An AI assistant skilled in summarizing and formatting information.",
    verbose=True,
    allow_delegation=False,
    llm="gemini/gemini-1.5-flash",
    tools=[]  # Writer does not require tools
)

## 3.Define Tasks

In [8]:
from crewai import Task

# Weather Report Task
weather_task = Task(
    description="Fetch and provide a detailed weather report for a specific location.",
    agent=weather_agent,
    expected_output="""A detailed weather report including temperature, humidity, 
    wind speed, and weather conditions."""
)

# Search Task
search_task = Task(
    description="Search for information about the human habits suitable for Today's weather conditions in the location.",
    agent=search_agent,
    expected_output="Search results about the human habits suitable for Today's weather trends and patterns in the location. Mention the Today's Date"
)

# Writing Task
writing_task = Task(
    description="""Summarize the information gathered by the Weather and Search agents 
    into a well-structured report, including insights and recommendations. But give more focus to Today's weather
    report and talks around it only.""",
    agent=writer_agent,
    expected_output="""A structured weather report summarizing the key findings, 
    including detailed weather conditions and relevant trends. Don't forget to mention today's Date."""
)


## 4. Create Crew and Execute

In [9]:
from crewai import Crew, Process

# Create Crew with Agents and Tasks
crew = Crew(
    agents=[weather_agent, search_agent, writer_agent],
    tasks=[weather_task, search_task, writing_task],
    # verbose=True, NOOO Never here
    process=Process.sequential
)


In [11]:
# Input: City/Location from User
# location = input("Enter the location for the weather report: ")
location = 'mumbai'

result = crew.kickoff(
    inputs={"location": location},
    # output_format="text"
)

# Agent: Weather Reporter
## Task: Fetch and provide a detailed weather report for a specific location.


# Agent: Weather Reporter
## Thought: tool_code
Thought: I need to use the my_simple_tool to get the current weather for Mumbai.
## Using tool: my_simple_tool
## Tool Input: 
"{\"location\": \"Mumbai\"}"
## Tool Output: 

        Weather in Mumbai:
        Country: India
        Region: Maharashtra
        Latitude: 18.975, Longitude: 72.826
        Local-Time: 2025-07-20 00:11

        Condition: Mist
        Temperature: 28.2°C / 82.8°F
        Feels Like: 33.0°C / 91.4°F
        Humidity: 89%
        Cloud Cover: 50%
        Precipitation: 0.14 mm
        Wind Speed: 19.1 km/h (11.9 mph)
        Wind Direction: SW (216°)
        Gust Speed: 27.6 km/h (17.2 mph)

        Visibility: 3.0 km
        UV Index: 0.0
        Pressure: 1003.0 mb / 29.62 in

        Weather Updated At: 2025-07-20 00:00
        


# Agent: Weather Reporter
## Final Answer: 
Weather in Mumbai:
Country: Ind